In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 

Using TensorFlow backend.


In [3]:
#datasetx = genfromtxt('feature.csv', delimiter='\t', skip_header=1)
#datasety = genfromtxt('output_y.csv', delimiter='\t', skip_header=1)

## Load the data

dataset is [datax, datay] before PCA  
dataset_PCA is that after PCA

In [4]:
datax = pd.read_csv('feature.csv', sep='\t')
datay = pd.read_csv('output_y.csv', sep='\t')

In [5]:
# merge x and y, and drop NA
pd_data=pd.concat([datax,datay],axis=1)
pd_data = pd_data.dropna(axis=0, how='any') 
#Remove bad data
pd_data = pd_data[~pd_data['discount_rate'].isin(['販売価格'])]
pd_data = pd_data[~pd_data['discount_rate'].isin(['ダミー'])]
#pick proper data, y<100
dataset = np.array(pd_data)
dataset = dataset[dataset[:,207]<100,:]
dataset_non_zero = dataset[dataset[:,207]!=0,:]

In [6]:
dataset.shape

(155335, 208)

In [7]:
dataset_non_zero.shape

(64188, 208)

## PCA

In [8]:
feature = dataset[:,0:207]

In [9]:
feature.shape

(155335, 207)

In [10]:
feature_dim = 207

In [11]:
pca=PCA(n_components=feature_dim)
feature_pca=pca.fit_transform(feature)

In [12]:
feature_pca.shape

(155335, 207)

In [13]:
dataset_pca = np.concatenate((feature_pca,dataset[:,207:208]),axis=1)
dataset_pca.shape

(155335, 208)

In [14]:
dataset_pca_non_zero = dataset_pca[dataset_pca[:,feature_dim]!=0,:]
dataset_pca_non_zero.shape

(64188, 208)

In [15]:
dataset_pca_non_zero.shape

(64188, 208)

## Model

In [10]:
feature_dim = 207

traindata = dataset[:20000]
testdata = dataset[20000:]

traindata_non_zero = dataset_non_zero[:12000]
testdata_non_zero = dataset_non_zero[12000:]

In [11]:
x_train, y_train = traindata[:,0:feature_dim], traindata[:,feature_dim:]
x_test, y_test = testdata[:,0:feature_dim], testdata[:,feature_dim:]

x_train_non_zero, y_train_non_zero = traindata_non_zero[:,0:feature_dim], traindata_non_zero[:,feature_dim:]
x_test_non_zero, y_test_non_zero = testdata_non_zero[:,0:feature_dim], testdata_non_zero[:,feature_dim:]

In [12]:
model = Sequential()

model.add(Dense(150, input_dim=feature_dim, init='normal', activation='relu')) 
#model.add(Dropout(0.2))

model.add(Dense(50, init='normal', activation='relu')) 
#model.add(Dropout(0.2))

model.add(Dense(1, init='normal')) 

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])

In [13]:
model_non_zero = Sequential()

model_non_zero.add(Dense(150, input_dim=feature_dim, init='normal', activation='relu')) 
#model.add(Dropout(0.2))

model_non_zero.add(Dense(50, init='normal', activation='relu')) 
#model.add(Dropout(0.2))

model_non_zero.add(Dense(1, init='normal')) 

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_non_zero.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])

## Traning and testing

### dataset y<100, y!=0

In [14]:
# training
print('Training -----------')
model_non_zero.fit(x_train_non_zero, y_train_non_zero, epochs=7, batch_size = 20)

Training -----------
Epoch 1/7
12000/12000 [==============================] - 1s 75us/step - loss: 443.4608 - mean_absolute_error: 13.5249
Epoch 2/7
12000/12000 [==============================] - 1s 68us/step - loss: 331.4277 - mean_absolute_error: 11.9551
Epoch 3/7
12000/12000 [==============================] - 1s 67us/step - loss: 310.5222 - mean_absolute_error: 11.6478
Epoch 4/7
12000/12000 [==============================] - 1s 66us/step - loss: 305.6025 - mean_absolute_error: 11.5396
Epoch 5/7
12000/12000 [==============================] - 1s 66us/step - loss: 302.1468 - mean_absolute_error: 11.4585
Epoch 6/7
12000/12000 [==============================] - 1s 66us/step - loss: 299.4324 - mean_absolute_error: 11.4206
Epoch 7/7
12000/12000 [==============================] - 1s 76us/step - loss: 296.9802 - mean_absolute_error: 11.3755


In [15]:
val_mse, val_mae = model_non_zero.evaluate(x_test_non_zero, y_test_non_zero, batch_size=20)
print ('mse=',val_mse, '\nmae=',val_mae)

52188/52188 [==============================] - 1s 20us/step
mse= 292.4511747755675 
mae= 12.663783947846035


In [16]:
intermediate_layer_model = Model(input=model_non_zero.input,
                                 output=model_non_zero.layers[2].output)
y_output_non_zero = intermediate_layer_model.predict(x_test_non_zero)
y_output_non_zero_pd = pd.DataFrame(y_output_non_zero)
y_test_non_zero_pd = pd.DataFrame(y_test_non_zero)

In [17]:
print('---------- y_test_non_zero ----------')
print('min=', np.min(y_test_non_zero))
print('max=', np.max(y_test_non_zero))
print('mean=', np.mean(y_test_non_zero))
print('median=', np.median(y_test_non_zero))
print('std=', np.std(y_test_non_zero))
print('var=', np.var(y_test_non_zero))

print('\n---------- y_output_non_zero ----------')
print('min=', np.min(y_output_non_zero))
print('max=', np.max(y_output_non_zero))
print('mean=', np.mean(y_output_non_zero))
print('median=', np.median(y_output_non_zero))
print('std=', np.std(y_output_non_zero))
print('var=', np.var(y_output_non_zero))

---------- y_test_non_zero ----------
min= 1
max= 99
mean= 12.092645818962213
median= 5.0
std= 17.575026711571724
var= 308.8815639124596

---------- y_output_non_zero ----------
min= 0.29922366
max= 76.73287
mean= 16.179586
median= 15.938181
std= 7.645302
var= 58.45064


### dataset y<100

In [20]:
# training
print('Training -----------')
model.fit(x_train, y_train, epochs=7, batch_size = 20)

Training -----------
Epoch 1/7
20000/20000 [==============================] - 1s 67us/step - loss: 5.6269 - mean_absolute_error: 0.0284
Epoch 2/7
20000/20000 [==============================] - 1s 67us/step - loss: 1695.5525 - mean_absolute_error: 0.3040
Epoch 3/7
20000/20000 [==============================] - 1s 69us/step - loss: 4.1218 - mean_absolute_error: 0.0264
Epoch 4/7
20000/20000 [==============================] - 1s 63us/step - loss: 777.9694 - mean_absolute_error: 0.2096
Epoch 5/7
20000/20000 [==============================] - 1s 68us/step - loss: 0.6627 - mean_absolute_error: 0.0139
Epoch 6/7
20000/20000 [==============================] - 1s 70us/step - loss: 573.9242 - mean_absolute_error: 0.1828
Epoch 7/7
20000/20000 [==============================] - 1s 67us/step - loss: 27.6274 - mean_absolute_error: 0.0491


In [21]:
val_mse, val_mae = model.evaluate(x_test, y_test, batch_size=20)
print ('mse=',val_mse, '\nmae=',val_mae)

135335/135335 [==============================] - 2s 18us/step
mse= 287.60044103119753 
mae= 5.886247474813724


In [24]:
intermediate_layer_model = Model(input=model.input,
                                 output=model.layers[2].output)
y_output = intermediate_layer_model.predict(x_test)
y_output_pd = pd.DataFrame(y_output)
y_test_pd = pd.DataFrame(y_test)

In [25]:
print('---------- y_test ----------')
print('min=', np.min(y_test))
print('max=', np.max(y_test))
print('mean=', np.mean(y_test))
print('median=', np.median(y_test))
print('std=', np.std(y_test))
print('var=', np.var(y_test))

print('\n---------- y_output ----------')
print('min=', np.min(y_output))
print('max=', np.max(y_output))
print('mean=', np.mean(y_output))
print('median=', np.median(y_output))
print('std=', np.std(y_output))
print('var=', np.var(y_output))

---------- y_test ----------
min= 0
max= 99
mean= 5.833272989248901
median= 0.0
std= 13.62077414809642
var= 185.52548839345172

---------- y_output ----------
min= -2185.8335
max= -0.0006866165
mean= -0.05297448
median= -0.0006866165
std= 8.247453
var= 68.02048
